# plotting returns: comparing stocks with indices

Related: [Plotting a stock's price chart with corporate action overlay](https://github.com/stockviz/plutons/blob/master/charting-stocks-with-corporate-actions.R.ipynb)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
equitiesIndiaNse <- EquitiesIndiaNse()
indices <- Indices()

In [ ]:
symbol <- "YESBANK"
indexName <- "NIFTY BANK TR"
startDt <- as.Date(Sys.Date() - 10*365) #10-year chart
endDt <- Sys.Date()

# examples: https://github.com/shyams80/plutons/blob/master/docs-R/EquitiesIndiaNse.ipynb

#make sure that the data exists

equitiesIndiaNse$EodTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt 
           & SYMBOL == symbol 
           & (SERIES == 'EQ' | SERIES == 'BE')) %>%
    summarize(MAX_DT = max(TIME_STAMP), MIN_DT = min(TIME_STAMP)) %>%
    print()

equitiesIndiaNse$EodAdjustedTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt 
           & SYMBOL == symbol) %>%
    summarize(MAX_DT = max(TIME_STAMP), MIN_DT = min(TIME_STAMP)) %>%
    print()


In [ ]:
# so the data exists in both so we can use the EodAdjustedTimeSeries

eodPx <- equitiesIndiaNse$EodAdjustedTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt & SYMBOL == symbol) %>%
    select(TIME_STAMP, CLOSE) %>%
    arrange(TIME_STAMP) %>%
    collect() 

indexPx <- indices$NseTimeSeries() %>%
    filter(NAME == indexName && TIME_STAMP >= startDt && TIME_STAMP <= endDt) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect() 

dailies <- merge(dailyReturn(xts(eodPx$CLOSE, as.Date(eodPx$TIME_STAMP))), 
                 dailyReturn(xts(indexPx$CLOSE, as.Date(indexPx$TIME_STAMP))))

In [ ]:
startDate <- first(index(dailies))
endDate <- last(index(dailies))

xAxisTicks <- seq(from=startDate, to=endDate, length.out=10)

Common.PlotCumReturns(dailies, sprintf("%s vs. %s", indexName, symbol), "")

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)